##Text to SQL using Prompt Engineering

- Create a employee dataset
- Create new column and update the details in it through the prompt

# **Assignment:** Adding a Employee Dataset

# Data Loading

In [4]:
# Original datasets from the first part of the project
! curl "https://api.mockaroo.com/api/dde01370?count=1000&key=11149690" > "customers.csv"
! curl "https://api.mockaroo.com/api/8ba6f630?count=1000&key=11149690" > "products.csv"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 99566    0 99566    0     0  65061      0 --:--:--  0:00:01 --:--:-- 65075
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  683k    0  683k    0     0   232k      0 --:--:--  0:00:02 --:--:--  232k


In [5]:
! curl "https://api.mockaroo.com/api/9bb4af90?count=200&key=12883420" > "Employees_Data.csv"
! curl "https://api.mockaroo.com/api/6fa67fe0?count=3000&key=11149690" > "orders.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18872    0 18872    0     0  17124      0 --:--:--  0:00:01 --:--:-- 17125
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  229k    0  229k    0     0  29662      0 --:--:--  0:00:07 --:--:-- 37052


In [6]:
!ls -l *.csv

-rw-r--r-- 1 root root  99566 Sep 27 03:28 customers.csv
-rw-r--r-- 1 root root  18872 Sep 27 03:28 Employees_Data.csv
-rw-r--r-- 1 root root 235124 Sep 27 03:28 orders.csv
-rw-r--r-- 1 root root 700226 Sep 27 03:28 products.csv


## Database Schema

We define the following tables:

- **customers**
- **products**
- **orders**
- **employees**


In [29]:
import sqlite3
import pandas as pd
import os
import numpy as np # Import numpy for random choice

customers_schema = """
CREATE TABLE IF NOT EXISTS customers (
    customer_id INT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50),
    email VARCHAR(50), phone_number VARCHAR(50), address VARCHAR(50),
    city VARCHAR(50), country VARCHAR(50), postal_code VARCHAR(50), loyalty_points INT
);
"""

products_schema = """
CREATE TABLE IF NOT EXISTS products (
    product_id INT PRIMARY KEY, product_name TEXT, description TEXT, price DECIMAL(10,2),
    discount_percentage DECIMAL(5,2), category VARCHAR(50), brand TEXT, stock_quantity INT,
    color VARCHAR(50), size VARCHAR(20), weight DECIMAL(5,2), dimensions TEXT,
    release_date DATE, rating DECIMAL(3,1), reviews_count INT, seller_name TEXT,
    seller_rating DECIMAL(3,1), seller_reviews_count INT, shipping_method VARCHAR(20),
    shipping_cost DECIMAL(6,2)
);
"""

orders_schema = """
CREATE TABLE IF NOT EXISTS orders (
    order_id INT PRIMARY KEY, customer_id INT, product_id INT, quantity INT,
    unit_price DECIMAL(10,2), total_price DECIMAL(10,2), order_date DATE,
    shipping_address VARCHAR(255), payment_method VARCHAR(20), status VARCHAR(20),
    employee_id INT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
);
"""

employees_schema = """
CREATE TABLE IF NOT EXISTS employees (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    job_title VARCHAR(50),
    department VARCHAR(50),
    hire_date DATE,
    salary INT,
    manager_id INT
);
"""


COLUMN_DATA_TYPES = {
    'customers': { 'customer_id': 'int64', 'loyalty_points': 'int64' },
    'products': { 'product_id': 'int64', 'price': 'float64', 'discount_percentage': 'float64', 'stock_quantity': 'int64', 'weight': 'float64', 'release_date': 'datetime64[ns]', 'rating': 'float64', 'reviews_count': 'int64', 'seller_rating': 'float64', 'seller_reviews_count': 'int64', 'shipping_cost': 'float64' },
    'orders': { 'order_id': 'int64', 'customer_id': 'int64', 'product_id': 'int64', 'quantity': 'int64', 'unit_price': 'float64', 'total_price': 'float64', 'order_date': 'datetime64[ns]', 'employee_id': 'int64' },
    'employees': { 'employee_id': 'int64', 'hire_date': 'datetime64[ns]', 'salary': 'int64', 'manager_id': 'Int64' } # Use Int64 for nullable integers
}

db_name = 'ecommerce_with_employees.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

conn = None
try:
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created.")

    # Create all tables
    cursor.execute(customers_schema)
    cursor.execute(products_schema)
    cursor.execute(employees_schema) # Create employees before orders due to foreign key
    cursor.execute(orders_schema)
    print("All tables created successfully.")

    # Map CSVs to table names and schemas
    csv_to_table_map = {
        'customers.csv': ('customers', COLUMN_DATA_TYPES['customers']),
        'products.csv': ('products', COLUMN_DATA_TYPES['products']),
        'orders.csv': ('orders', COLUMN_DATA_TYPES['orders']), # Corrected filename here
        'Employees_Data.csv': ('employees', COLUMN_DATA_TYPES['employees'])
    }

    # Load employees first to get employee_ids
    employees_df = None
    if os.path.exists('Employees_Data.csv'):
        try:
            employees_df = pd.read_csv('Employees_Data.csv')
            for col, dtype in COLUMN_DATA_TYPES['employees'].items():
                 if col in employees_df.columns:
                    if 'datetime' in dtype:
                        employees_df[col] = pd.to_datetime(employees_df[col], errors='coerce')
                    else:
                        try:
                            if dtype == 'Int64':
                                employees_df[col] = pd.to_numeric(employees_df[col], errors='coerce').astype('Int64')
                            else:
                                employees_df[col] = pd.to_numeric(employees_df[col], errors='coerce').astype(dtype)
                        except (ValueError, TypeError) as e:
                             print(f"Error converting column '{col}' in employees_df: {e}")
            employees_df.to_sql('employees', conn, if_exists='append', index=False)
            print("  -> Data from 'Employees_Data.csv' loaded into 'employees'.")
        except Exception as e:
            print(f"Error loading data from 'Employees_Data.csv' into 'employees': {e}")


    for csv_file, (table_name, dtypes) in csv_to_table_map.items():
        if table_name == 'employees':
            continue # Already handled employees loading
        if os.path.exists(csv_file):
            try:
                df = pd.read_csv(csv_file)
                print(f"Attempting to load data from '{csv_file}' into '{table_name}'.")
                print(f"Columns in CSV: {df.columns.tolist()}")
                print(f"Expected dtypes: {dtypes}")
                for col, dtype in dtypes.items():
                    if col in df.columns:
                        if 'datetime' in dtype:
                            df[col] = pd.to_datetime(df[col], errors='coerce')
                        else:
                            # Attempt conversion and handle potential errors more specifically
                            try:
                                if dtype == 'Int64': # Handle nullable integer
                                     df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
                                else:
                                     df[col] = pd.to_numeric(df[col], errors='coerce').astype(dtype)

                            except ValueError as ve:
                                print(f"ValueError converting column '{col}' to '{dtype}' in '{csv_file}': {ve}")
                            except TypeError as te:
                                print(f"TypeError converting column '{col}' to '{dtype}' in '{csv_file}': {te}")
                    else:
                        print(f"Warning: Column '{col}' not found in '{csv_file}'.")

                # Special handling for orders to add employee_id
                if table_name == 'orders' and employees_df is not None:
                    if 'employee_id' not in df.columns:
                        print("Adding placeholder 'employee_id' to orders data.")
                        # Assign random employee_ids from the loaded employees
                        if not employees_df.empty:
                             df['employee_id'] = np.random.choice(employees_df['employee_id'].dropna().tolist(), size=len(df))
                        else:
                            print("Warning: employees_df is empty, cannot assign random employee_ids to orders.")
                            df['employee_id'] = None # Assign None if no employees are loaded

                df.to_sql(table_name, conn, if_exists='append', index=False)
                print(f"  -> Data from '{csv_file}' loaded into '{table_name}'.")
            except Exception as e:
                print(f"Error loading data from '{csv_file}' into '{table_name}': {e}")
        else:
            print(f"Warning: '{csv_file}' not found.")

    conn.commit()
    print("\nData committed successfully.")

except Exception as e:
    print(f"An error occurred during database setup or initial loading: {e}")
finally:
    if conn:
        conn.close()
        print("Database connection closed.")

Removed existing database 'ecommerce_with_employees.db'.
Database 'ecommerce_with_employees.db' created.
All tables created successfully.
  -> Data from 'Employees_Data.csv' loaded into 'employees'.
Attempting to load data from 'customers.csv' into 'customers'.
Columns in CSV: ['customer_id', 'first_name', 'last_name', 'email', 'phone_number', 'address', 'city', 'country', 'postal_code', 'loyalty_points']
Expected dtypes: {'customer_id': 'int64', 'loyalty_points': 'int64'}
  -> Data from 'customers.csv' loaded into 'customers'.
Attempting to load data from 'products.csv' into 'products'.
Columns in CSV: ['product_id', 'product_name', 'description', 'price', 'discount_percentage', 'category', 'brand', 'stock_quantity', 'color', 'size', 'weight', 'dimensions', 'release_date', 'rating', 'reviews_count', 'seller_name', 'seller_rating', 'seller_reviews_count', 'shipping_method', 'shipping_cost']
Expected dtypes: {'product_id': 'int64', 'price': 'float64', 'discount_percentage': 'float64', '

# Setting up an API

In [8]:
!pip install google-genai
from google.colab import userdata
import os
import json
import google.generativeai as genai

# Use userdata.get to access the API key from Colab secrets
genai_client = genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

# Prompt

In [9]:
updated_prompt = """
### **ROLE**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are the core translation engine for a business intelligence dashboard. The database dialect is always **SQLite**.

The database consists of the following four tables:

**customers table:**
```sql
CREATE TABLE customers (
    customer_id INT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50),
    email VARCHAR(50), phone_number VARCHAR(50), address VARCHAR(50),
    city VARCHAR(50), country VARCHAR(50), postal_code VARCHAR(50), loyalty_points INT
);
```

** products table:**
```sql
CREATE TABLE products (
    product_id INT PRIMARY KEY, product_name TEXT, description TEXT, price DECIMAL(10,2),
    discount_percentage DECIMAL(5,2), category VARCHAR(50), brand TEXT, stock_quantity INT,
    color VARCHAR(50), size VARCHAR(20), weight DECIMAL(5,2), dimensions TEXT,
    release_date DATE, rating DECIMAL(3,1), reviews_count INT, seller_name TEXT,
    seller_rating DECIMAL(3,1), seller_reviews_count INT, shipping_method VARCHAR(20),
    shipping_cost DECIMAL(6,2)
);
```

**orders table:**
```sql
CREATE TABLE orders (
    order_id INT PRIMARY KEY, customer_id INT, product_id INT, quantity INT,
    unit_price DECIMAL(10,2), total_price DECIMAL(10,2), order_date DATE,
    shipping_address VARCHAR(255), payment_method VARCHAR(20), status VARCHAR(20),
    employee_id INT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
);
```

**employees table:**
```sql
CREATE TABLE employees (
    employee_id INT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50),
    email VARCHAR(50), job_title VARCHAR(50), department VARCHAR(50),
    hire_date DATE, salary INT, manager_id INT
);
```

-----

### **TASK**

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1.  **Analyze the User's Query:** Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like `SUM`, `COUNT`, `AVG`), and ordering they are asking for.
2.  **Map to the Schema:** Map the entities from the user's query to the appropriate tables (`customers`, `products`, `orders`) and columns. Determine the necessary `JOIN` operations using `customers.customer_id` and `products.product_id` as foreign keys in the `orders` table.
3.  **Construct the SQLite Query:** Write a clean and efficient `SELECT` statement that is syntactically correct for SQLite. Ensure all table and column names are accurate.
4.  **Handle Ambiguity:** If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### **EXAMPLES**

**Example 1:**

  * **User Query:** "What are the names of the top 3 products with the highest total revenue?"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT T2.product_name FROM orders AS T1 INNER JOIN products AS T2 ON T1.product_id = T2.product_id GROUP BY T2.product_name ORDER BY SUM(T1.total_price) DESC LIMIT 3;"
    }
    ```

**Example 2: **

  * **User Query:** "Who is the highest paid employee?"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT first_name, last_name, salary FROM employees ORDER BY salary DESC LIMIT 1;"
    }
    ```

-----

### **OUTPUT FORMAT**

Your final response must be a single JSON object with two keys:

1.  `"status"`: A string with one of three possible values: `"success"`, `"clarification_needed"`, or `"error"`.
2.  `"response": A string containing the SQL query, clarifying question, or error message.

"""


In [14]:
import sqlite3
import pandas as pd
import json # Import json
import google.generativeai as genai # Import genai

def get_sql_query(genai_client, model_name, prompt, user_query):
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  # Use genai.GenerativeModel() instead of genai.get_model()
  model = genai.GenerativeModel(model_name)
  response = model.generate_content(contents)

  clean_response = response.text.replace('```json', '').replace('```', '').strip()
  try:
    output = json.loads(clean_response)
  except json.JSONDecodeError:
    print(f"ERROR: The AI returned a non-JSON response: \n{clean_response}\n---")
    output = {"status": "error", "response": "The AI model did not return a valid JSON format."}
  return output

def execute_query(query, db_name='ecommerce_with_employees.db'):

    conn = None
    try:
        conn = sqlite3.connect(db_name)
        print(f"\nExecuting query on '{db_name}':\n{query}")
        results_df = pd.read_sql_query(query, conn)
        print("Query executed successfully.")
        return results_df
    except Exception as e:
        print(f"Error executing query: {e}")
        return None
    finally:
        if conn:
            conn.close()

def text2sql(genai_client, model_name, prompt, user_query):

  print(f"Translating your question: '{user_query}'")
  output = get_sql_query(genai_client, model_name, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  else:
    print(f"\nReceived a non-success status: {output['status']}")
    print(f"Response: {output['response']}")
    return None

# Asking Questions

In [27]:
print("Running your Text-to-SQL query")
user_question = "Who are the top 5 highest-paid employees?"
result = text2sql(genai_client, 'models/gemini-2.0-flash', updated_prompt, user_question)

result

Running your Text-to-SQL query
Translating your question: 'Who are the top 5 highest-paid employees?'

Executing query on 'ecommerce_with_employees.db':
SELECT first_name, last_name FROM employees ORDER BY salary DESC LIMIT 5;
Query executed successfully.


,first_name,last_name
0,Jolie,Bee
1,Meredeth,Zanicchi
2,Ami,Balleine
3,Mathew,Awin
4,Myrtice,Priestland


In [30]:
print("Running your Text-to-SQL query")
user_question = "Which employee is responsible for the highest value of sales? Show their name and the total sales amount."
result = text2sql(genai_client, 'models/gemini-2.0-flash', updated_prompt, user_question)

result

Running your Text-to-SQL query
Translating your question: 'Which employee is responsible for the highest value of sales? Show their name and the total sales amount.'

Executing query on 'ecommerce_with_employees.db':
SELECT T2.first_name, T2.last_name, SUM(T1.total_price) AS total_sales_amount FROM orders AS T1 INNER JOIN employees AS T2 ON T1.employee_id = T2.employee_id GROUP BY T1.employee_id ORDER BY total_sales_amount DESC LIMIT 1;
Query executed successfully.


,first_name,last_name,total_sales_amount
0,Lexi,Zanolli,153940.99


In [32]:
print("Running your Text-to-SQL query")
user_question = "What are the total sales for each product category."
result = text2sql(genai_client, 'models/gemini-2.0-flash', updated_prompt, user_question)

result

Running your Text-to-SQL query
Translating your question: 'What are the total sales for each product category.'

Executing query on 'ecommerce_with_employees.db':
SELECT category, SUM(total_price) FROM products AS T1 INNER JOIN orders AS T2 ON T1.product_id = T2.product_id GROUP BY category
Query executed successfully.


,category,SUM(total_price)
0,beauty,2624335.52
1,clothing,2532605.14
2,electronics,3512828.32
3,home decor,3396672.06
4,toys,2853617.44


In [33]:
print("Running your Text-to-SQL query")
user_question = "Which products have a stock quantity below 100."
result = text2sql(genai_client, 'models/gemini-2.0-flash', updated_prompt, user_question)

result

Running your Text-to-SQL query
Translating your question: 'Which products have a stock quantity below 100.'

Executing query on 'ecommerce_with_employees.db':
SELECT product_name FROM products WHERE stock_quantity < 100;
Query executed successfully.


,product_name
0,in est risus auctor sed tristique in tempus si...
1,tempus sit amet sem fusce consequat nulla nisl...
2,vestibulum vestibulum ante ipsum primis in fau...
3,augue a suscipit nulla elit ac nulla sed vel e...
4,augue quam sollicitudin vitae consectetuer ege...
...,...
84,mauris vulputate elementum nullam varius nulla...
85,consequat dui nec nisi volutpat eleifend donec...
86,rutrum at lorem integer tincidunt ante vel ips...
87,nibh in hac habitasse platea dictumst aliquam ...
